# 1.Installation and settings

In [ ]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu

# download saved vector database
#gdown --folder 1ej1-cIitZFitlXChtUoGfFONwL8yIsXA

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 38.6 MB/s eta 0:00:00
Processing file 1-D3XELIFrvYg1tvHCS0pixJ2r_O-xfFR index.faiss
Processing file 1-Bxl5W5j69RqGPZO86hn9DjvZDmq0sPm index.pkl


Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-D3XELIFrvYg1tvHCS0pixJ2r_O-xfFR
To: /content/faiss_index_2/index.faiss
100%|██████████| 9.05M/9.05M [00:00<00:00, 84.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-Bxl5W5j69RqGPZO86hn9DjvZDmq0sPm
To: /content/faiss_index_2/index.pkl
100%|██████████| 5.01M/5.01M [00:00<00:00, 31.1MB/s]
Download completed


In [ ]:
import os
from google.colab import userdata # we stored our access token as a colab secret

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('UV_TOKEN')

# 2.Setting up LLM

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

# This info's at the top of each HuggingFace model page
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"


llm = HuggingFaceEndpoint(
    repo_id = hf_model,
    # max_new_tokens=512,
    temperature=1,
    top_p=0.99,
    repetition_penalty=1.03
)

# 3.Loading the data

In [ ]:
# Upload/drag-and-drop a pdf to a content folder bofore
file_path = (
    "../../content/spain.pdf"
)

In [ ]:
%pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
# Step 1: Load the PDF
pdf_loader = PyPDFLoader(file_path)
documents = pdf_loader.load()

In [ ]:
print(documents[100].page_content)

MADRID & AROUND NEIGHBOURHOODS AT A GLANCEMADRID & AROUND SIGHTS MADRID & AROUND SIGHTS
modest, examples of Romanesque architec-
ture in Madrid. When it was built, Madrid 
was a small village more than 2km away.
 Plaza de la Cibeles SQUARE
(Map p88;  mBanco de España)  Of all the 
grand roundab7outs that punctuate the 
Paseo del Prado,  Plaza de la Cibeles most 
evokes the splendour of imperial Madrid. 
The jewel in the crown is the astonishing 
Palacio de Comunicaciones (p99). Other 
landmark buildings around the plaza’s pe-
rimeter include the  Palacio de Linares and 
Casa de América  (Map p88; %91 595 48 00, 
ticket reservations 902 221424; www.casamerica.
es; Plaza de la  Cibeles 2; adult/student & senior/
child €8/5/free; hguided tours 11am, noon & 1pm 
Sat & Sun Sep-Jul, shorter hours Aug, ticket office 
10am-3pm & 4-8pm Mon-Fri; mBanco de España), 
the  Palacio Buenavista (Map p102; Plaza de 
la Cibeles; mBanco de España) and the national 
 Banco de España  ( Map p88; Calle de A

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                               chunk_overlap=150)

docs = text_splitter.split_documents(documents)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS

vector_db = FAISS.from_documents(docs, embeddings)

In [ ]:
vector_db.save_local("/content/faiss_index_3")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy the file to Google Drive
!cp -r /content/faiss_index_3 /content/drive/MyDrive/

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

In [ ]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2})
template = """You are a nice chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short and succinct.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:"""

chat_history = []

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

doc_retriever = create_history_aware_retriever(
    llm, retriever, prompt
)

doc_chain = create_stuff_documents_chain(llm, prompt)

rag_bot = create_retrieval_chain(
    doc_retriever, doc_chain
)

In [ ]:
ans = rag_bot.invoke({"input":"What is the capital of Spain?", "chat_history": chat_history, "context": retriever})
chat_history.extend([{"role": "human", "content": ans["input"]},{"role": "assistant", "content":ans["answer"]}])

In [ ]:
print(ans['answer'])


Assistant: Madrid.


In [ ]:
ans = rag_bot.invoke({"input": "What is the most popular street there?", "chat_history": chat_history, "context": retriever})
chat_history.extend([{"role": "human", "content": ans["input"]},{"role": "assistant", "content":ans["answer"]}])
print(ans["answer"])


AI: Gran Vía is one of the most popular streets in Madrid.


In [ ]:
ans = rag_bot.invoke({"input": "Are there some good cafes on this street?", "chat_history": chat_history, "context": retriever})
chat_history.extend([{"role": "human", "content": ans["input"]},{"role": "assistant", "content":ans["answer"]}])
print(ans["answer"])


AI: There are plenty of cafes along Gran Vía, but one popular option is the Café Oriente, located at number 25. It's known for its beautiful Art Nouveau decor and expansive terrace. Another option is El Corte Inglés, a large department store with several floors, including a café with views of the city.


In [ ]:
ans = rag_bot.invoke({"input": "Which museums should I visit in Valencia?", "chat_history": chat_history, "context": retriever})
chat_history.extend([{"role": "human", "content": ans["input"]},{"role": "assistant", "content":ans["answer"]}])
print(ans["answer"])


AI: In Valencia, I recommend the Museo Arqueológico de Alicante (MARQ) and the Ocean Race Museum. The MARQ offers a strong collection of ceramics and Iberian art, along with high-quality temporary exhibitions. The Ocean Race Museum showcases the history of the annual sailing race that starts in Valencia. Additionally, you can get a pass to visit multiple museums in the city.


In [ ]:
ans = rag_bot.invoke({"input": "How can I get there from Madrid?", "chat_history": chat_history, "context": retriever})
chat_history.extend([{"role": "human", "content": ans["input"]},{"role": "assistant", "content":ans["answer"]}])
print(ans["answer"])


AI: 
AI: You can travel to Valencia from Madrid by bus or train. Avanza operates regular bus services to/from Madrid (€28 to €36, 4¼ hours). ALSA has services to/from Barcelona (€29 to €38, four to six hours, up to 10 daily) and Alicante (€21 to €25, 2½ to 5½ hours, more than 10 daily), most via Benidorm. For train travel, all fast trains now use the Valencia Joaquín Sorolla station, located beside the riverbed park. It’s meant to be temporary, but looks like sticking around for a long time. It’s linked with nearby Estación del Norte, 500m away, by free shuttle bus. Estación del Norte has slow trains to Gandia, Alicante, Barcelona and Madrid, as well as swift AVEs.


In [ ]:
ans = rag_bot.invoke({"input": "Are there regular flights between these two cities?", "chat_history": chat_history, "context": retriever})
chat_history.extend([{"role": "human", "content": ans["input"]},{"role": "assistant", "content":ans["answer"]}])
print(ans["answer"])


AI: Yes, both Air Europa and Iberia offer regular flights between Madrid and Valencia.
